In [1]:
# ライブラリのインポート
import numpy as np
import pandas as pd
import itertools
import shutil
from PIL import Image
import os

import torch
from torch import nn, utils, optim
from torchvision import transforms, models
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
# GPUを使うかどうか
USE_DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [3]:
# データがあるディレクトリ
INPUT_DIR = '../input/forestpathmoviedataset/'

In [4]:
# PyTorchの内部を決定論的に設定する
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# 乱数を初期化する関数
def init_seed():
    np.random.seed(0)
    torch.manual_seed(0)

In [5]:
# 一時ディレクトリを作成
if not os.path.isdir('tmp'):
    os.mkdir('tmp')

In [6]:
# データセットの定義ファイルを読み込む
df = pd.read_csv(INPUT_DIR + 'all_file.csv')

In [7]:
# シーン毎に分割するので、groupbyして取り出す
file, person = [], []
for g in df.groupby(df['scene']):
    file.append(g[1].file.values.tolist())
    person.append(g[1].person.values.tolist())

#print(file)
#print(person)

In [8]:
# シーン毎に学習用と評価用データに分ける
train_X, test_X, train_y, test_y = train_test_split(file, person, test_size=0.3, random_state=0)

In [9]:
# 全てのシーン内のデータを繋げた配列にする
train_X = sum(train_X, [])
train_y = sum(train_y, [])
test_X = sum(test_X, [])
test_y = sum(test_y, [])

In [10]:
# データセットをクラスで定義する
class MyDataset:
    def __init__(self, file_name_list, label_list, valid=False):
        # 初期化
        self.file_name_list = file_name_list
        self.label_list = label_list
        if not valid: # 学習用ならデータの水増しを含んだtransformを作る
            transform = [
                transforms.Resize((224,224)),
                transforms.ColorJitter(brightness=1.0),
                transforms.RandomGrayscale(0.1),
                transforms.ToTensor(),
                transforms.RandomErasing(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
            ]
        else: # 評価時にはデータの水増しを含まないtransformを作る
            transform = [
                transforms.Resize((224,224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
            ]
        self.transform = transforms.Compose(transform)

    def __len__(self):
        # データセットの長さを返す
        return len(self.file_name_list)

    def __getitem__(self, index):
        # indexの場所にあるデータを返す
        image_path = INPUT_DIR + self.file_name_list[index] # ファイルパス
        img = Image.open(image_path) # ファイルを読み込む
        img = self.transform(img) # データ水増ししてtensorにする
        label = self.label_list[index]
        return img, label

In [11]:
def get_model(): # ニューラルネットワークのモデルを返す関数
    
    # ModelZOOからモデルをダウンロードして最後の層だけを入れ替える
    model = models.resnet50(pretrained=True)
    model.fc = nn.Linear(2048, 2) # 出力の数＝2にする
    model = model.to(USE_DEVICE) # GPUを使うときはGPUメモリ上に乗せる
    
    return model

In [12]:
def get_optim(model, lr): # 勾配降下法のアルゴリズムを返す関数
    params = model.parameters() # 学習させるパラメーター
    optimizer = optim.SGD(params, lr=lr,  momentum=0.9) # 学習率を設定
    return optimizer

In [13]:
def get_loss(weight): # 損失関数を返す関数
    
    # 不均衡なデータを学習させるために、クラス毎のウェイトを設定する
    weight = torch.tensor([1.0-weight,weight], dtype=torch.float)
    weight = weight.to(USE_DEVICE) # GPUを使うときはGPUメモリ上に乗せる
    loss = nn.CrossEntropyLoss(weight=weight) # ウェイト付きのCrossEntropy
    
    return loss

In [14]:
def get_score(true_valid, pred_valid): # 評価スコアを返す関数
    
    # 評価スコアは、全体の他に時間帯毎にも作成するのでディクショナリを用意する
    timezone = {'daytime':([],[]),'twilight':([],[]),'midnight':([],[])}
    
    # 認識結果を時間帯毎に仕分けする
    for i, filename in enumerate(test_X):
        w = df[df['file']==filename].when.values[0]
        timezone[w][0].append(true_valid[i])
        timezone[w][1].append(pred_valid[i])
    
    # 時間帯毎のF1スコアをディクショナリに入れる
    score = {k:f1_score(v[0], v[1]) for k,v in timezone.items()}
    
    # 全体のF1スコアをディクショナリに入れる
    score['total'] = f1_score(true_valid, pred_valid)
    
    # 認識が極端に偏ってないか見るために、認識値の平均も求める
    score['average'] = np.mean(pred_valid)
    
    return score

In [15]:
# 学習時と評価時のバッチサイズ
BATCH_SIZE = 16
BATCH_SIZE_VALID = 4

# データの読み込みスレッドの数
NUM_WORKERS = 2

# 試行時の学習エポック数
NUM_EPOCHS = 5

# 評価で試す学習率
LR_TESTS = [1e-3, 2e-4, 5e-5]

# 試すウェイトは、人物の方が分散が大きいので、クラス1側を0.5より少なくする
WEIGHT_TESTS = [0.1, 0.2, 0.3, 0.4, 0.5]

In [16]:
# 学習用と評価用にデータセットを作る
train_dataset = MyDataset(train_X, train_y)
test_dataset = MyDataset(test_X, test_y, True)

In [17]:
# 複数スレッドでファイルを読み込みつつデータを取り出すDataLoaderを作る
train_dataloader = utils.data.DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)

valid_dataloader = utils.data.DataLoader(
    test_dataset, batch_size=BATCH_SIZE_VALID, shuffle=False, num_workers=NUM_WORKERS)

In [18]:
# 試行時の最も評価が良かったスコアのリスト
best_scores = []

# 学習率とウェイトを変えながら試行する
for t, (lr, weight) in enumerate(itertools.product(LR_TESTS, WEIGHT_TESTS)):
    # 試行毎に乱数を初期化してからニューラルネットワークを作成する
    init_seed()
    model = get_model() # ニューラルネットワークを作成
    # 学習のためのアルゴリズムを取得
    optimizer = get_optim(model, lr)
    loss = get_loss(weight)

    # 現在の学習率とウェイトで試行する
    print(f'test #{t} lr={lr} weight={weight}')
    print('========================')
    
    scores = [] # 各エポック終了時のスコア

    # 学習ループ
    for epoch in range(NUM_EPOCHS):
        total_loss = [] # 各バッチ実行時の損失値
        model.train() # モデルを学習用に設定する
        for images, labels in train_dataloader: # 画像を読み込んでtensorにする
            images = images.to(USE_DEVICE) # GPUを使うときはGPUメモリ上に乗せる
            labels = labels.to(USE_DEVICE) # GPUを使うときはGPUメモリ上に乗せる

            # ニューラルネットワークを実行して損失値を求める
            losses = loss(model(images), labels)

            # 新しいバッチ分の学習を行う
            optimizer.zero_grad() # 一つ前の勾配をクリア
            losses.backward() # 損失値を逆伝播させる
            optimizer.step() # 新しい勾配からパラメーターを更新する

            # 損失値を保存しておく
            total_loss.append(losses.detach().cpu().numpy())

        # 評価
        with torch.no_grad():
            # 評価時の損失値と正解/認識結果を入れるリスト
            total_loss_v = []
            true_valid = []
            pred_valid = []

            model.eval() # モデルを推論用に設定する
            for i, (images, labels) in enumerate(valid_dataloader):
                images = images.to(USE_DEVICE) # GPUを使うときはGPUメモリ上に乗せる
                labels = labels.to(USE_DEVICE) # GPUを使うときはGPUメモリ上に乗せる

                outputs = model(images) # ニューラルネットワークの実行
                losses = loss(outputs, labels) # 評価データの損失値

                # 正解データを保存
                labels = labels.detach().cpu().numpy() # CPUメモリに入れてnumpy化
                true_valid.extend(labels.tolist())
                # 認識結果を保存
                outputs = outputs.detach().cpu().numpy() # CPUメモリに入れてnumpy化
                pred_valid.extend(outputs.argmax(axis=1).tolist())

                # 損失値を保存しておく
                total_loss_v.append(losses.detach().cpu().numpy())

        # エポック終了時のスコアを求める
        total_loss = np.mean(total_loss) # 各バッチの損失の平均
        total_loss_v = np.mean(total_loss_v) # 各バッチの損失の平均
        score = get_score(true_valid, pred_valid) # 評価スコア
        scores.append(score['total']) # スコアを保存しておく
        # エポック終了時のスコアを表示する
        print(f'epoch #{epoch+1}: train_loss:{total_loss:.4f} valid_loss:{total_loss_v:.4f} score:{score}')
        print('--------------------------------------------')
        # エポック終了時のモデルを保存しておく
        torch.save(model.state_dict(), f'tmp/checkpoint{epoch}.pth')

    # 現在の学習率とウェイトで最も良かったモデルをコピーして保存しておく
    best_epoch = np.argmax(scores)
    shutil.copyfile(f'tmp/checkpoint{best_epoch}.pth',f'tmp/test{t}_best.pth')
    # 現在の学習率とウェイトで最も良かったモデルを損失値しておく
    best_scores.append(scores[best_epoch])

    # GPUメモリをGCする
    del model, optimizer, loss, images, labels, outputs, losses
    torch.cuda.empty_cache()

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

test #0 lr=0.001 weight=0.1
epoch #1: train_loss:0.2285 valid_loss:0.7494 score:{'daytime': 0.9158730158730158, 'twilight': 0.4869325997248969, 'midnight': 0.6593607305936073, 'total': 0.7854263102881888, 'average': 0.509406392694064}
--------------------------------------------
epoch #2: train_loss:0.1126 valid_loss:0.6142 score:{'daytime': 0.8810469735919608, 'twilight': 0.6568986568986569, 'midnight': 0.9088256746900073, 'total': 0.8673469387755103, 'average': 0.7181735159817352}
--------------------------------------------
epoch #3: train_loss:0.0850 valid_loss:0.9189 score:{'daytime': 0.7732974910394266, 'twilight': 0.6634382566585957, 'midnight': 0.9094858797972484, 'total': 0.8087431693989071, 'average': 0.7568949771689498}
--------------------------------------------
epoch #4: train_loss:0.0723 valid_loss:1.0777 score:{'daytime': 0.806332682715246, 'twilight': 0.59514687100894, 'midnight': 0.9159970781592404, 'total': 0.8229217904574521, 'average': 0.7715068493150685}
---------

In [19]:
# 最も良かった学習率とウェイトでのモデルをコピーする
best_of_best = np.argmax(best_scores)
shutil.copyfile(f'tmp/test{best_of_best}_best.pth', '/kaggle/working/best-model.pth')

'/kaggle/working/best-model.pth'

In [20]:
# 一時ディレクトリを削除
shutil.rmtree('tmp')